## Gathering Data from RIOT API 

Importing relevant libraries to pull and organize the data

In [21]:
import requests
import time
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

Setting the API key variable here

In [2]:
key = 'RGAPI-2c26df9d-9e4b-427f-bbba-2f300e15b217'

We set two variables retrieving static data from the API that was not require an API key. This will be our base data that we use from here on to clean and convert values.

In [26]:
champion_data = requests.get('http://ddragon.leagueoflegends.com/cdn/10.16.1/data/en_US/champion.json').json()
rank = requests.get(f'https://na1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={key}').json()
spells = requests.get('http://ddragon.leagueoflegends.com/cdn/10.16.1/data/en_US/summoner.json').json()
items = requests.get('http://ddragon.leagueoflegends.com/cdn/10.16.1/data/en_US/item.json').json()

From the summoner rank json, we want to extract the Summoner ID for every player retrieved and store them into a list

In [4]:
master_rank_summoner_id = [i['summonerId'] for i in rank['entries']]

We want to change into another api reference and using the summoner ID we retrieved previously, we want to now retrieve the
encrypted account ID and store those into another list. From here on, the sleep timer will be 1.3 seconds because Riot (company that owns League of Legends) has a limit of the calls for their API. Since we have a development key, we are only allowed to request a max of 100 calls per 120 seconds. That is 1 call per 1.2 seconds but we wanted to give some room for rest so we set it to 1.3 seconds per call.

In [7]:
master_rank_account_id = []

for i in master_rank_summoner_id:
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + str(i) + '?api_key=' + key
    master_rank_account_id.append(requests.get(url).json()['accountId'])
    time.sleep(1.3)

In [8]:
with open("../files/master_rank_account_id.txt", "w") as output:
    output.write(str(master_rank_account_id))

At this step, we want the match ID which is requires another layer of digging through the API reference. With the last piece
of the puzzle at hand, we use the account IDs to grab the match IDs from every player and store that information into another
list

In [118]:
master_rank_match_id = []

for i in master_rank_account_id:
    url = 'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/' + i + '?api_key=' + key
    try:
        data = requests.get(url).json()
        for i in data['matches']:
            master_rank_match_id.append(i['gameId'])
    except:
        pass
    time.sleep(1.3)

After all the loops, we do not want to repeat that process again, therefore we saved a text file containing all the match ID
data so we can just call up the IDs without pulling all over again from the top.

In [121]:
with open("../files/master_rank_match_id.txt", "w") as output:
    output.write(str(master_rank_match_id))

This function opens the text file we generated but because the text file was saved as a single string, we use the eval()
function to convert the information back into a list

In [4]:
with open('../files/master_rank_match_id.txt', 'r') as f:
    temp = f.read().splitlines()

match_id = eval(temp[0])

As a precaution, we want to remove any duplicate data before we retrieve any player information. this can be simply applied
but setting the list into a set then back into a list to ensure no duplicates remain

In [5]:
match_id = list(set(match_id))

Finally, we have our match data information and we begin pulling player data per match. PLEASE KEEP IN MIND THIS WILL TAKE A 
VERY LONG TIME. The file containing all player data is stored in the data folder so no need to pull again.

In [6]:
match_data = []

for i in match_id:
    url = f'https://na1.api.riotgames.com/lol/match/v4/matches/{str(i)}?api_key={key}'
    match_data.append(requests.get(url).json())
    time.sleep(1.25)

Since the data retrieve is a huge dictionary, we only want specific data for our analysis. Below we set a placeholder for the dictionary to hold the information we want and append that information to a list created outside the loop. We do this for all players in the file then set it into a dataframe below.

In [7]:
player_list = []
for match in range(len(match_data)):
    for items in range(5):
        try:
            player_dict_item = {}
            if match_data[match].get('participants'):
                player_dict_item['game_mode'] = match_data[match].get('gameMode')
            player_dict_item['champion'] = match_data[match].get('participants')[items].get('championId')
            player_dict_item['spell_1'] = match_data[match].get('participants')[items].get('spell1Id')
            player_dict_item['spell_2'] = match_data[match].get('participants')[items].get('spell2Id')
            player_dict_item['item_0'] = match_data[match].get('participants')[items]['stats'].get('item0')
            player_dict_item['item_1'] = match_data[match].get('participants')[items]['stats'].get('item1')
            player_dict_item['item_2'] = match_data[match].get('participants')[items]['stats'].get('item2')
            player_dict_item['item_3'] = match_data[match].get('participants')[items]['stats'].get('item3')
            player_dict_item['item_4'] = match_data[match].get('participants')[items]['stats'].get('item4')
            player_dict_item['item_5'] = match_data[match].get('participants')[items]['stats'].get('item5')
            player_dict_item['item_6'] = match_data[match].get('participants')[items]['stats'].get('item6')
            player_dict_item['victory'] = match_data[match].get('participants')[items]['stats'].get('win')
            player_dict_item['kills'] = match_data[match].get('participants')[items]['stats'].get('kills')
            player_dict_item['deaths'] = match_data[match].get('participants')[items]['stats'].get('deaths')
            player_dict_item['assists'] = match_data[match].get('participants')[items]['stats'].get('assists')
            player_dict_item['largest_killing_spree'] = match_data[match].get('participants')[items]['stats'].get('largestKillingSpree')
            player_dict_item['killing_sprees'] = match_data[match].get('participants')[items]['stats'].get('killingSprees')
            player_dict_item['largest_time_spent_living'] = match_data[match].get('participants')[items]['stats'].get('longestTimeSpentLiving')
            player_dict_item['double_kills'] = match_data[match].get('participants')[items]['stats'].get('doubleKills')
            player_dict_item['triple_kills'] = match_data[match].get('participants')[items]['stats'].get('tripleKills')
            player_dict_item['quadra_kills'] = match_data[match].get('participants')[items]['stats'].get('quadraKills')
            player_dict_item['penta_kills'] = match_data[match].get('participants')[items]['stats'].get('pentaKills')
            player_dict_item['total_heal'] = match_data[match].get('participants')[items]['stats'].get('totalHeal')
            player_dict_item['total_units_healed'] = match_data[match].get('participants')[items]['stats'].get('totalUnitsHealed')
            player_dict_item['damage_self_mitigated'] = match_data[match].get('participants')[items]['stats'].get('damageSelfMitigated')
            player_dict_item['total_damage_dealt_to_champions'] = match_data[match].get('participants')[items]['stats'].get('totalDamageDealtToChampions')
            player_dict_item['total_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('totalDamageDealt')
            player_dict_item['magical_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('magicDamageDealt')
            player_dict_item['phsical_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('physicalDamageDealt')
            player_dict_item['true_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('trueDamageDealt') 
            player_dict_item['vision_score'] = match_data[match].get('participants')[items]['stats'].get('visionScore')
            player_dict_item['total_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('totalDamageTaken')
            player_dict_item['magical_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('magicalDamageTaken')
            player_dict_item['physical_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('physicalDamageTaken')
            player_dict_item['true_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('trueDamageTaken')
            player_dict_item['gold_earned'] = match_data[match].get('participants')[items]['stats'].get('goldEarned')
            player_dict_item['gold_spent'] = match_data[match].get('participants')[items]['stats'].get('goldSpent')   
            player_dict_item['turret_kills'] = match_data[match].get('participants')[items]['stats'].get('turretKills')
            player_dict_item['inhibitor_kills'] = match_data[match].get('participants')[items]['stats'].get('inhibitorKills')
            player_dict_item['total_minions_killed'] = match_data[match].get('participants')[items]['stats'].get('totalMinionsKilled')
            player_dict_item['wards_placed'] = match_data[match].get('participants')[items]['stats'].get('wardsPlaced')
            player_dict_item['wards_killed'] = match_data[match].get('participants')[items]['stats'].get('wardsKilled')   
            player_dict_item['first_blood_kill'] = match_data[match].get('participants')[items]['stats'].get('firstBloodKill')
            player_dict_item['first_blood_assist'] = match_data[match].get('participants')[items]['stats'].get('firstBloodAssist')
            player_dict_item['first_tower_kill'] = match_data[match].get('participants')[items]['stats'].get('firstTowerKill')
            player_dict_item['first_tower_assist'] = match_data[match].get('participants')[items]['stats'].get('firstTowerAssist')
            player_dict_item['first_inhibitor_kill'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorKill')
            player_dict_item['first_inhibitor_assist'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorAssist')  
            player_dict_item['lane'] = match_data[match].get('participants')[items]['timeline'].get('lane')
            player_dict_item['role'] = match_data[match].get('participants')[items]['timeline'].get('role')
            player_list.append(player_dict_item)
            
        except:
            pass
        
    for items in range(5,10):
        try:
            player_dict_item = {}
            player_dict_item['game_mode'] = match_data[match].get('gameMode')
            if match_data[match].get('participants')[items]:
                player_dict_item['champion'] = match_data[match].get('participants')[items].get('championId')
            player_dict_item['spell_1'] = match_data[match].get('participants')[items].get('spell1Id')
            player_dict_item['spell_2'] = match_data[match].get('participants')[items].get('spell2Id')
            player_dict_item['item_0'] = match_data[match].get('participants')[items]['stats'].get('item0')
            player_dict_item['item_1'] = match_data[match].get('participants')[items]['stats'].get('item1')
            player_dict_item['item_2'] = match_data[match].get('participants')[items]['stats'].get('item2')
            player_dict_item['item_3'] = match_data[match].get('participants')[items]['stats'].get('item3')
            player_dict_item['item_4'] = match_data[match].get('participants')[items]['stats'].get('item4')
            player_dict_item['item_5'] = match_data[match].get('participants')[items]['stats'].get('item5')
            player_dict_item['item_6'] = match_data[match].get('participants')[items]['stats'].get('item6')
            player_dict_item['victory'] = match_data[match].get('participants')[items]['stats'].get('win')
            player_dict_item['kills'] = match_data[match].get('participants')[items]['stats'].get('kills')
            player_dict_item['deaths'] = match_data[match].get('participants')[items]['stats'].get('deaths')
            player_dict_item['assists'] = match_data[match].get('participants')[items]['stats'].get('assists')
            player_dict_item['largest_killing_spree'] = match_data[match].get('participants')[items]['stats'].get('largestKillingSpree')
            player_dict_item['killing_sprees'] = match_data[match].get('participants')[items]['stats'].get('killingSprees')
            player_dict_item['largest_time_spent_living'] = match_data[match].get('participants')[items]['stats'].get('longestTimeSpentLiving')
            player_dict_item['double_kills'] = match_data[match].get('participants')[items]['stats'].get('doubleKills')
            player_dict_item['triple_kills'] = match_data[match].get('participants')[items]['stats'].get('tripleKills')
            player_dict_item['quadra_kills'] = match_data[match].get('participants')[items]['stats'].get('quadraKills')
            player_dict_item['penta_kills'] = match_data[match].get('participants')[items]['stats'].get('pentaKills')
            player_dict_item['total_heal'] = match_data[match].get('participants')[items]['stats'].get('totalHeal')
            player_dict_item['total_units_healed'] = match_data[match].get('participants')[items]['stats'].get('totalUnitsHealed')
            player_dict_item['damage_self_mitigated'] = match_data[match].get('participants')[items]['stats'].get('damageSelfMitigated')
            player_dict_item['total_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('totalDamageDealt')
            player_dict_item['magical_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('magicDamageDealt')
            player_dict_item['phsical_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('physicalDamageDealt')
            player_dict_item['true_damage_dealt'] = match_data[match].get('participants')[items]['stats'].get('trueDamageDealt')
            player_dict_item['total_damage_dealt_to_champions'] = match_data[match].get('participants')[items]['stats'].get('totalDamageDealtToChampions')
            player_dict_item['vision_score'] = match_data[match].get('participants')[items]['stats'].get('visionScore')
            player_dict_item['total_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('totalDamageTaken')
            player_dict_item['magical_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('magicalDamageTaken')
            player_dict_item['physical_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('physicalDamageTaken')
            player_dict_item['true_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('trueDamageTaken')
            player_dict_item['gold_earned'] = match_data[match].get('participants')[items]['stats'].get('goldEarned')
            player_dict_item['gold_spent'] = match_data[match].get('participants')[items]['stats'].get('goldSpent')   
            player_dict_item['turret_kills'] = match_data[match].get('participants')[items]['stats'].get('turretKills')
            player_dict_item['inhibitor_kills'] = match_data[match].get('participants')[items]['stats'].get('inhibitorKills')
            player_dict_item['total_minions_killed'] = match_data[match].get('participants')[items]['stats'].get('totalMinionsKilled')
            player_dict_item['wards_placed'] = match_data[match].get('participants')[items]['stats'].get('wardsPlaced')
            player_dict_item['wards_killed'] = match_data[match].get('participants')[items]['stats'].get('wardsKilled')   
            player_dict_item['first_blood_kill'] = match_data[match].get('participants')[items]['stats'].get('firstBloodKill')
            player_dict_item['first_blood_assist'] = match_data[match].get('participants')[items]['stats'].get('firstBloodAssist')
            player_dict_item['first_tower_kill'] = match_data[match].get('participants')[items]['stats'].get('firstTowerKill')
            player_dict_item['first_tower_assist'] = match_data[match].get('participants')[items]['stats'].get('firstTowerAssist')
            player_dict_item['first_inhibitor_kill'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorKill')
            player_dict_item['first_inhibitor_assist'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorAssist')  
            player_dict_item['lane'] = match_data[match].get('participants')[items]['timeline'].get('lane')
            player_dict_item['role'] = match_data[match].get('participants')[items]['timeline'].get('role')
            player_list.append(player_dict_item)
        except:
            pass


This step simply stores the giant list we created into a dataframe

In [8]:
df = pd.DataFrame(player_list)

As a precaution, we stored the dataframe as a `.csv` file in case we have to restart the notebook and do not want to pull the data again. For the data pulling from Riot's API, it was split into 3 parts since the API key expires in 24 hours. Each part below was data pulled from 8000 match ID at a time.

In [15]:
# df.to_csv('../data/part1.csv')

In [33]:
# df.to_csv('../data/part2.csv')

In [10]:
# df.to_csv('../data/part3.csv')

Here we want to import the 3 parts previously stored files from the `data` folder.

In [17]:
part_1 = pd.read_csv('../data/part1.csv', index_col = 0)
part_2 = pd.read_csv('../data/part2.csv', index_col = 0)
part_3 = pd.read_csv('../data/part3.csv', index_col = 0)

In [18]:
part_3.head()

,assists,champion,damage_self_mitigated,deaths,double_kills,first_blood_assist,first_blood_kill,first_inhibitor_assist,first_inhibitor_kill,first_tower_assist,...,total_minions_killed,total_units_healed,triple_kills,true_damage_dealt,true_damage_taken,turret_kills,victory,vision_score,wards_killed,wards_placed
0,4,245,10362,2,1,False,False,NaN,NaN,False,...,31,1,0,3608,838,0,True,13,2.0,4.0
1,6,81,2895,3,0,False,True,NaN,NaN,False,...,134,1,0,452,66,3,True,12,2.0,7.0
2,9,43,4223,1,0,False,False,NaN,NaN,False,...,34,5,0,3480,48,0,True,42,4.0,17.0
3,3,164,12677,1,0,False,False,NaN,NaN,False,...,121,1,0,8489,8,1,True,12,0.0,7.0
4,4,134,3827,2,1,False,False,NaN,NaN,False,...,124,1,0,3875,95,0,True,23,4.0,9.0


In [33]:
final = pd.concat([part_1, part_2, part_3])

As above, we want to save another `csv` file just in case we get interrupted creating the data.

In [23]:
# final.to_csv('../data/final.csv')

At this step, we want to incorporate some formatting before we bring the data into main notebook. The champions column are numerical IDs instead of their actual name. From the data we pull above, we have all the champion information including their name and associated ID. We first create a list of dictionary where the ID is the key and value is the name. Then we store that information into a list for every champion. Next we do the same for summoner spells. The file was a `json` file downloaded from Riot's [Data Dragon](https://developer.riotgames.com/ddragon.html) page. Similar to what we did with the character list, we created a list of dictionarys with the key being the ID and value being the spell name.

In [27]:
champion_dict = {champion_data['data'][list(champion_data['data'].keys())[i]]['key']: list(champion_data['data'].keys())[i]  
                 for i 
                 in range(len(list(champion_data['data'].keys())))}


spell_dict = {spells['data'][list(spells['data'].keys())[i]]['key']: spells['data'][list(spells['data'].keys())[i]]['name'] 
             for i
             in range(len(list(spells['data'].keys())))}

items_dict = {list(items['data'])[i]: items['data'][list(items['data'])[i]]['name'] 
              for i 
              in range(len(list(items['data'])))}

We converted the following columns into string in preparation of replacing all values with the appropriate names.

In [40]:
columns = ['champion', 'spell_1', 'spell_2', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6']

for col in columns:
    final[col] = final[col].astype(str)

Here we map the IDs to their names and replace every value in the data with the appropriate name.

In [42]:
final['champion'] = final['champion'].map(champion_dict)
final['spell_1'] = final['spell_1'].map(spell_dict)
final['spell_2'] = final['spell_2'].map(spell_dict)

for col in columns[3:]:
    final[col] = final[col].map(items_dict)

We finally save the file ready for analysis in our main notebook.

In [43]:
final.to_csv('../data/final_with_data.csv')

Here continue [here](League%20of%20Legends.ipynb) for full analysis of the data.